In [1]:
import pandas as pd
import numpy as np

loans = pd.read_csv('lending-club-data.csv')
loans.head()

/home/qiangwennorge/anaconda3/lib/python3.6/site-packages/IPython/core/interactiveshell.py:2698: DtypeWarning: Columns (19,47) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


,id,member_id,loan_amnt,funded_amnt,funded_amnt_inv,term,int_rate,installment,grade,sub_grade,...,sub_grade_num,delinq_2yrs_zero,pub_rec_zero,collections_12_mths_zero,short_emp,payment_inc_ratio,final_d,last_delinq_none,last_record_none,last_major_derog_none
0,1077501,1296599,5000,5000,4975,36 months,10.65,162.87,B,B2,...,0.4,1.0,1.0,1.0,0,8.14350,20141201T000000,1,1,1
1,1077430,1314167,2500,2500,2500,60 months,15.27,59.83,C,C4,...,0.8,1.0,1.0,1.0,1,2.39320,20161201T000000,1,1,1
2,1077175,1313524,2400,2400,2400,36 months,15.96,84.33,C,C5,...,1.0,1.0,1.0,1.0,0,8.25955,20141201T000000,1,1,1
3,1076863,1277178,10000,10000,10000,36 months,13.49,339.31,C,C1,...,0.2,1.0,1.0,1.0,0,8.27585,20141201T000000,0,1,1
4,1075269,1311441,5000,5000,5000,36 months,7.90,156.46,A,A4,...,0.8,1.0,1.0,1.0,0,5.21533,20141201T000000,1,1,1


In [2]:
loans['safe_loans'] = loans['bad_loans'].apply(lambda x : +1 if x==0 else -1)
loans = loans.drop(['bad_loans'], axis = 1)

In [3]:
features = ['grade',              # grade of the loan
            'term',               # the term of the loan
            'home_ownership',     # home_ownership status: own, mortgage or rent
            'emp_length',         # number of years of employment
           ]
target = 'safe_loans'

In [4]:
import json

train_idx = json.loads(open('module-5-assignment-2-train-idx.json').read())
test_idx = json.loads(open('module-5-assignment-2-test-idx.json').read())

train_data = loans.iloc[train_idx]
test_data = loans.iloc[test_idx]
train_data.head()

,id,member_id,loan_amnt,funded_amnt,funded_amnt_inv,term,int_rate,installment,grade,sub_grade,...,delinq_2yrs_zero,pub_rec_zero,collections_12_mths_zero,short_emp,payment_inc_ratio,final_d,last_delinq_none,last_record_none,last_major_derog_none,safe_loans
1,1077430,1314167,2500,2500,2500,60 months,15.27,59.83,C,C4,...,1.0,1.0,1.0,1,2.39320,20161201T000000,1,1,1,-1
6,1071795,1306957,5600,5600,5600,60 months,21.28,152.39,F,F2,...,1.0,1.0,1.0,0,4.57170,20161201T000000,1,1,1,-1
7,1071570,1306721,5375,5375,5350,60 months,12.69,121.45,B,B5,...,1.0,1.0,1.0,1,9.71600,20161201T000000,1,1,1,-1
10,1064687,1298717,9000,9000,9000,36 months,13.49,305.38,C,C1,...,1.0,1.0,1.0,1,12.21520,20141201T000000,1,1,1,-1
12,1069057,1303503,10000,10000,10000,36 months,10.65,325.74,B,B2,...,1.0,1.0,1.0,0,3.90888,20141201T000000,1,1,1,-1


### Subsample dataset to make sure classes are balanced

Just as we did in the previous assignment, we will undersample the larger class (safe loans) in order to balance out our dataset. This means we are throwing away many data points. You should have code analogous to

In [5]:
safe_loans_raw = loans[loans[target] == 1]
risky_loans_raw = loans[loans[target] == -1]

# Since there are less risky loans than safe loans, find the ratio of the sizes
# and use that percentage to undersample the safe loans.
percentage = len(risky_loans_raw)/float(len(safe_loans_raw))
safe_loans = safe_loans_raw.sample(frac = percentage, random_state = 1)
risky_loans = risky_loans_raw
loans_data = risky_loans.append(safe_loans)

print("Percentage of safe loans                 :", len(safe_loans) / float(len(loans_data)))
print("Percentage of risky loans                :", len(risky_loans) / float(len(loans_data)))
print("Total number of loans in our new dataset :", len(loans_data))

Percentage of safe loans                 : 0.5
Percentage of risky loans                : 0.5
Total number of loans in our new dataset : 46300


### Transform categorical data into binary features

In this assignment, we will implement binary decision trees (decision trees for binary features, a specific case of categorical variables taking on two values, e.g., true/false). Since all of our features are currently categorical features, we want to turn them into binary features.

For instance, the home_ownership feature represents the home ownership status of the loanee, which is either own, mortgage or rent. For example, if a data point has the feature

In [6]:
grade_val = list(loans_data['grade'].unique())
len(grade_val)

7

In [7]:
term_val = list(loans_data['term'].unique())
len(term_val)

2

In [8]:
home_ownership_val = list(loans_data['home_ownership'].unique())
len(home_ownership_val)

4

In [9]:
emp_length_val = list(loans_data['emp_length'].unique())
len(emp_length_val)

12

In [10]:
loans_data_new = loans_data[target].reset_index()

for feature in features:
    loans_data_one_hot_encoded = loans_data[feature].apply(lambda x: {x: 1})    
    loans_data_unpacked = pd.DataFrame(list(loans_data_one_hot_encoded)).fillna(0)   
    

    loans_data_new = pd.concat([loans_data_new, loans_data_unpacked], axis = 1)

#loans_data_unpacked
loans_data = loans_data_new.drop(['index'], axis = 1)
loans_data.head()

,safe_loans,A,B,C,D,E,F,G,36 months,60 months,...,2 years,3 years,4 years,5 years,6 years,7 years,8 years,9 years,< 1 year,n/a
0,-1,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
1,-1,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,...,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,-1,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
3,-1,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
4,-1,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,...,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


### Train-test split

Split the data into a train test split with 80% of the data in the training set and 20% of the data in the test set. Call these datasets train_data and test_data, respectively. 

In [11]:
from sklearn.model_selection import train_test_split
train_x, test_x, train_y, test_y = train_test_split(loans_data.drop([target], axis = 1), loans_data[target], test_size = 0.2, random_state = 1)

### Function to count number of mistakes while predicting majority class

Recall from the lecture that prediction at an intermediate node works by predicting the majority class for all data points that belong to this node. Now, we will write a function that calculates the number of misclassified examples when predicting the majority class. This will be used to help determine which feature is the best to split on at a given node of the tree.

Note: Keep in mind that in order to compute the number of mistakes for a majority classifier, we only need the label (y values) of the data points in the node.

Steps to follow:

 *   Step 1: Calculate the number of safe loans and risky loans.
 *   Step 2: Since we are assuming majority class prediction, all the data points that are not in the majority class are considered mistakes.
 *   Step 3: Return the number of mistakes.

Now, let us write the function intermediate_node_num_mistakes which computes the number of misclassified examples of an intermediate node given the set of labels (y values) of the data points contained in the node. Your code should be analogous to

In [12]:
def intermediate_node_num_mistakes(labels_in_node):
    # Corner case: If labels_in_node is empty, return 0
    if len(labels_in_node) == 0:
        return 0    
    # Count the number of 1's (safe loans)
    num_of_positive = (labels_in_node == +1).sum()  
    # Count the number of -1's (risky loans)
    num_of_negative = (labels_in_node == -1).sum()        
    # Return the number of mistakes that the majority classifier makes.
    return num_of_negative if num_of_positive > num_of_negative else num_of_positive

Because there are several steps in this assignment, we have introduced some stopping points where you can check your code and make sure it is correct before proceeding. To test your intermediate_node_num_mistakes function, run the following code until you get a Test passed!, then you should proceed. Otherwise, you should spend some time figuring out where things went wrong. Again, remember that this code is specific to SFrame, but using your software of choice, you can construct similar tests.

In [13]:
# Test case 1
example_labels = np.array([-1, -1, 1, 1, 1])
if intermediate_node_num_mistakes(example_labels) == 2:
    print('Test passed!')
else:
    print('Test 1 failed... try again!')

# Test case 2
example_labels = np.array([-1, -1, 1, 1, 1, 1, 1])
if intermediate_node_num_mistakes(example_labels) == 2:
    print('Test passed!')
else:
    print('Test 3 failed... try again!')
    
# Test case 3
example_labels = np.array([-1, -1, -1, -1, -1, 1, 1])
if intermediate_node_num_mistakes(example_labels) == 2:
    print('Test passed!')
else:
    print('Test 3 failed... try again!')

Test passed!
Test passed!
Test passed!


### Function to pick best feature to split on

The function best_splitting_feature takes 3 arguments:

  *  The data
  *  The features to consider for splits (a list of strings of column names to consider for splits)
  *  The name of the target/label column (string)

The function will loop through the list of possible features, and consider splitting on each of them. It will calculate the classification error of each split and return the feature that had the smallest classification error when split on.

Recall that the classification error is defined as follows:

classification error=# mistakes / # total examples

Follow these steps to implement best_splitting_feature:

  *  Step 1: Loop over each feature in the feature list
  *  Step 2: Within the loop, split the data into two groups: one group where all of the data has feature value 0 or False (we will call this the left split), and one group where all of the data has feature value 1 or True (we will call this the right split). Make sure the left split corresponds with 0 and the right split corresponds with 1 to ensure your implementation fits with our implementation of the tree building process.
  *  Step 3: Calculate the number of misclassified examples in both groups of data and use the above formula to compute theclassification error.
  *  Step 4: If the computed error is smaller than the best error found so far, store this feature and its error.

Note: Remember that since we are only dealing with binary features, we do not have to consider thresholds for real-valued features. This makes the implementation of this function much easier.

Your code should be analogous to

In [14]:
def best_splitting_feature(data, features, target):
    
    target_values = data[target]
    best_feature = None # Keep track of the best feature 
    best_error = 10     # Keep track of the best error so far 
    # Note: Since error is always <= 1, we should intialize it with something larger than 1.

    # Convert to float to make sure error gets computed correctly.
    num_data_points = float(len(data))  
    
    # Loop through each feature to consider splitting on that feature
    for feature in features:
        
        # The left split will have all data points where the feature value is 0
        left_split = data[data[feature] == 0]
        
        # The right split will have all data points where the feature value is 1
        right_split = data[data[feature] == 1] 
            
        # Calculate the number of misclassified examples in the left split.
        # Remember that we implemented a function for this! (It was called intermediate_node_num_mistakes)
        left_mistakes = intermediate_node_num_mistakes(left_split[target])            

        # Calculate the number of misclassified examples in the right split.
        right_mistakes = intermediate_node_num_mistakes(right_split[target])
            
        # Compute the classification error of this split.
        # Error = (# of mistakes (left) + # of mistakes (right)) / (# of data points)

        error = (left_mistakes + right_mistakes) / num_data_points

        # If this is the best error we have found so far, store the feature as best_feature and the error as best_error
        if error < best_error:
            best_feature = feature
            best_error = error        
    
    return best_feature # Return the best feature we found

### Building the tree

With the above functions implemented correctly, we are now ready to build our decision tree. Each node in the decision tree is represented as a dictionary which contains the following keys and possible values:

{ 

   'is_leaf'            : True/False.
   
   'prediction'         : Prediction at the leaf node.
   
   'left'               : (dictionary corresponding to the left tree).
   
   'right'              : (dictionary corresponding to the right tree).
   
   'splitting_feature'  : The feature that this node splits on
   
}

First, we will write a function that creates a leaf node given a set of target values. Your code should be analogous to

In [15]:
def create_leaf(target_values):    
    # Create a leaf node
    leaf = {'splitting_feature' : None,
            'left' : None,
            'right' : None,
            'is_leaf': True}   
   
    # Count the number of data points that are +1 and -1 in this node.
    num_ones = len(target_values[target_values == +1])
    num_minus_ones = len(target_values[target_values == -1])    

    # For the leaf node, set the prediction to be the majority class.
    # Store the predicted class (1 or -1) in leaf['prediction']
    if num_ones > num_minus_ones:
        leaf['prediction'] = 1
    else:
        leaf['prediction'] = -1           

    # Return the leaf node
    return leaf 

We have provided a function that learns the decision tree recursively and implements 3 stopping conditions:

  *  Stopping condition 1: All data points in a node are from the same class.
  *  Stopping condition 2: No more features to split on.
  *  Additional stopping condition: In addition to the above two stopping conditions covered in lecture, in this assignment we will also consider a stopping condition based on the max_depth of the tree. By not letting the tree grow too deep, we will save computational effort in the learning process.

Now, we will provide a Python skeleton of the learning algorithm. Note that this code is not complete; it needs to be completed by you if you are using Python. Otherwise, your code should be analogous to

In [17]:
def decision_tree_create(data, features, target, current_depth = 0, max_depth = 10):
    remaining_features = features[:] # Make a copy of the features.
    
    target_values = data[target]
    print("--------------------------------------------------------------------")
    print("Subtree, depth = %s (%s data points)." % (current_depth, len(target_values)))
    

    # Stopping condition 1
    # (Check if there are mistakes at current node.
    # Recall you wrote a function intermediate_node_num_mistakes to compute this.)
    if intermediate_node_num_mistakes(target_values) == 0: 
        print("Stopping condition 1 reached.")  
        # If not mistakes at current node, make current node a leaf node
        return create_leaf(target_values)
    
    # Stopping condition 2 (check if there are remaining features to consider splitting on)
    if remaining_features == []:   
        print("Stopping condition 2 reached.")   
        # If there are no remaining features to consider, make current node a leaf node
        return create_leaf(target_values)    
    
    # Additional stopping condition (limit tree depth)
    if current_depth >= max_depth:  
        print("Reached maximum depth. Stopping for now.")
        # If the max tree depth has been reached, make current node a leaf node
        return create_leaf(target_values)

    # Find the best splitting feature (recall the function best_splitting_feature implemented above)
    splitting_feature = best_splitting_feature(data, features, target)
    
    # Split on the best feature that we found. 
    left_split = data[data[splitting_feature] == 0]
    right_split = data[data[splitting_feature] == 1]
    remaining_features.remove(splitting_feature)
    print("Split on feature %s. (%s, %s)" % (\
                      splitting_feature, len(left_split), len(right_split)))
    
    # Create a leaf node if the split is "perfect"
    if len(left_split) == len(data):
        print("Creating leaf node.")
        return create_leaf(left_split[target])
    if len(right_split) == len(data):
        print("Creating leaf node.")
        return create_leaf(right_split[target])

        
    # Repeat (recurse) on left and right subtrees
    left_tree = decision_tree_create(left_split, remaining_features, target, current_depth + 1, max_depth)        

    right_tree = decision_tree_create(right_split, remaining_features, target, current_depth + 1, max_depth)

    return {'is_leaf'          : False, 
            'prediction'       : None,
            'splitting_feature': splitting_feature,
            'left'             : left_tree, 
            'right'            : right_tree}

### Build the tree!

Train a tree model on the train_data. Limit the depth to 6 (max_depth = 6) to make sure the algorithm doesn't run for too long. Call this tree my_decision_tree. Warning: The tree may take 1-2 minutes to learn.

In [18]:
train_data = pd.concat([train_x, train_y], axis = 1)
features = list(train_x.columns.values)

my_decision_tree = decision_tree_create(train_data, features, 'safe_loans', 0, 6)

--------------------------------------------------------------------
Subtree, depth = 0 (37040 data points).
Split on feature  36 months. (9419, 27621)
--------------------------------------------------------------------
Subtree, depth = 1 (9419 data points).
Split on feature A. (9293, 126)
--------------------------------------------------------------------
Subtree, depth = 2 (9293 data points).
Split on feature B. (8227, 1066)
--------------------------------------------------------------------
Subtree, depth = 3 (8227 data points).
Split on feature C. (5974, 2253)
--------------------------------------------------------------------
Subtree, depth = 4 (5974 data points).
Split on feature D. (3891, 2083)
--------------------------------------------------------------------
Subtree, depth = 5 (3891 data points).
Split on feature E. (1721, 2170)
--------------------------------------------------------------------
Subtree, depth = 6 (1721 data points).
Reached maximum depth. Stopping for 

### Making predictions with a decision tree

As discussed in the lecture, we can make predictions from the decision tree with a simple recursive function. Write a function called classify, which takes in a learned tree and a test point x to classify. Include an option annotate that describes the prediction path when set to True. Your code should be analogous to

In [19]:
def classify(tree, x, annotate = False):
       # if the node is a leaf node.
    if tree['is_leaf']:
        if annotate:
             print("At leaf, predicting %s" % tree['prediction'])
        return tree['prediction']
    else:
        # split on feature.
        split_feature_value = x[tree['splitting_feature']]
        if annotate:
             print("Split on %s = %s" % (tree['splitting_feature'], split_feature_value))
        if split_feature_value == 0:
            return classify(tree['left'], x, annotate)
        else:
            return classify(tree['right'], x, annotate)

Now, let's consider the first example of the test set and see what my_decision_tree model predicts for this data point.

In [20]:
test_data = pd.concat([test_x, test_y], axis = 1)
print('Actual class   : %s ' % int(test_data.iloc[0][-1]))
print('Predicted class: %s ' % classify(my_decision_tree, test_data.iloc[0]))

Actual class   : -1 
Predicted class: -1 


# Question 1
What was the feature that my_decision_tree first split on while making the prediction for test_data[0]?

In [65]:
classify(my_decision_tree, test_data.iloc[0], annotate=True)

Split on  36 months = 1.0
Split on D = 0.0
Split on E = 1.0
At leaf, predicting -1


-1

* term. 36 months

# Question 2
What was the first feature that lead to a right split of test_data[0]?

* grade.D

# Question 3
What was the last feature split on before reaching a leaf node for test_data[0]?

* grade.D

### Evaluating your decision tree

Now, we will write a function to evaluate a decision tree by computing the classification error of the tree on the given dataset. Write a function called evaluate_classification_error that takes in as input:

 *   tree (as described above)
 *   data (a data frame of data points)

This function should return a prediction (class label) for each row in data using the decision tree. Your code should be analogous to

In [29]:
def evaluate_classification_error(tree, data):
    # Apply the classify(tree, x) to each row in your data
    prediction = data.apply(lambda x: classify(tree, x), axis = 1)
    
    # Once you've made the predictions, calculate the classification error and return it
    error = (prediction != data[target]).sum()/float(len(data))
    return error

Now, use this function to evaluate the classification error on the test set.

# Question 4
Rounded to 2nd decimal point (e.g. 0.76), what is the classification error of my_decision_tree on the test_data?

In [31]:
evaluate_classification_error(my_decision_tree, test_data)

0.382829373650108

### Printing out a decision stump

As discussed in the lecture, we can print out a single decision stump (printing out the entire tree is left as an exercise to the curious reader). Here we provide Python code to visualize a decision stump. If you are using different software, make sure your code is analogous to:

In [35]:
def print_stump(tree, name = 'root'):
    split_name = tree['splitting_feature'] # split_name is something like 'term. 36 months'
    if split_name is None:
        print("(leaf, label: %s)" % tree['prediction'])
        return None
    #split_feature, split_value = split_name.split('.')
    print('                       %s' % name)
    print('         |---------------|----------------|')
    print('         |                                |')
    print('         |                                |')
    print('         |                                |')
    print('  [{0} == 0]               [{0} == 1]    '.format(split_name))
    print('         |                                |')
    print('         |                                |')
    print('         |                                |')
    print('    (%s)                         (%s)' \
        % (('leaf, label: ' + str(tree['left']['prediction']) if tree['left']['is_leaf'] else 'subtree'),
           ('leaf, label: ' + str(tree['right']['prediction']) if tree['right']['is_leaf'] else 'subtree')))

Using this function, we can print out the root of our decision tree:

In [36]:
print_stump(my_decision_tree)

                       root
         |---------------|----------------|
         |                                |
         |                                |
         |                                |
  [ 36 months == 0]               [ 36 months == 1]    
         |                                |
         |                                |
         |                                |
    (subtree)                         (subtree)


# Question 5
What is the feature that is used for the split at the root node?

* term. 36 months

### Exploring the intermediate left subtree

The tree is a recursive dictionary, so we do have access to all the nodes! We can use

 *   my_decision_tree['left'] to go left
 *   my_decision_tree['right'] to go right

We can print out the left subtree by running the code

In [37]:
print_stump(my_decision_tree['left'], my_decision_tree['splitting_feature'])

                        36 months
         |---------------|----------------|
         |                                |
         |                                |
         |                                |
  [A == 0]               [A == 1]    
         |                                |
         |                                |
         |                                |
    (subtree)                         (subtree)


We can similarly print out the left subtree of the left subtree of the root by running the code

In [38]:
print_stump(my_decision_tree['left']['left'], my_decision_tree['left']['splitting_feature'])

                       A
         |---------------|----------------|
         |                                |
         |                                |
         |                                |
  [B == 0]               [B == 1]    
         |                                |
         |                                |
         |                                |
    (subtree)                         (subtree)


# Question 6
What is the path of the first 3 feature splits considered along the left-most branch of my_decision_tree?

* term. 36 months, grade.A, grade.B

# Question 7
What is the path of the first 3 feature splits considered along the right-most branch of my_decision_tree?

In [39]:
print_stump(my_decision_tree['right'], my_decision_tree['splitting_feature'])

                        36 months
         |---------------|----------------|
         |                                |
         |                                |
         |                                |
  [D == 0]               [D == 1]    
         |                                |
         |                                |
         |                                |
    (subtree)                         (leaf, label: -1)


In [40]:
print_stump(my_decision_tree['right']['right'], my_decision_tree['right']['splitting_feature'])

(leaf, label: -1)


* term. 36 months, grade.D, no third feature because second split resulted in leaf